In [55]:
import pandas as pd

df = pd.read_csv('heart.csv')
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [56]:
import numpy as np
from scipy.stats import zscore

df_numeric = df.select_dtypes(include = [np.number])

zscores = np.abs(zscore(df_numeric))

threshold = 3

filtered = (zscores<threshold).all(axis = 1)

df_1 = df[filtered]
df_1.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [57]:
df_2 = pd.get_dummies(df_1, dtype='int')
df_2.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_F,Sex_M,ChestPainType_ASY,...,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,0,1,0,...,0,0,0,1,0,1,0,0,0,1
1,49,160,180,0,156,1.0,1,1,0,0,...,1,0,0,1,0,1,0,0,1,0
2,37,130,283,0,98,0.0,0,0,1,0,...,0,0,0,0,1,1,0,0,0,1
3,48,138,214,0,108,1.5,1,1,0,1,...,0,0,0,1,0,0,1,0,1,0
4,54,150,195,0,122,0.0,0,0,1,0,...,1,0,0,1,0,1,0,0,0,1


In [58]:
X= df_2.drop('HeartDisease', axis = 1)
y = df_2['HeartDisease']

In [59]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[-1.42815446,  0.46590022,  0.84963584, ..., -0.26018448,
        -0.99888827,  1.13469459],
       [-0.47585532,  1.63471366, -0.16812204, ..., -0.26018448,
         1.00111297, -0.88129441],
       [-1.7455875 , -0.1185065 ,  0.79361247, ..., -0.26018448,
        -0.99888827,  1.13469459],
       ...,
       [ 0.3706328 , -0.1185065 , -0.62564622, ..., -0.26018448,
         1.00111297, -0.88129441],
       [ 0.3706328 , -0.1185065 ,  0.35476274, ..., -0.26018448,
         1.00111297, -0.88129441],
       [-1.63977649,  0.34901888, -0.21480818, ..., -0.26018448,
        -0.99888827,  1.13469459]])

In [60]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 5)

In [61]:
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC
scores = cross_val_score(SVC(), X_train, y_train, cv = 5)
scores.mean()

0.8623251748251748

In [62]:
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier(
    estimator = SVC(),
    n_estimators = 100,
    max_samples = 0.8,
    oob_score = True,
    random_state = 5)

scores_1 = cross_val_score(model, X_train, y_train, cv = 5) 
scores_1.mean()

0.8623251748251748

In [63]:
from sklearn.tree import DecisionTreeClassifier

model_1 = DecisionTreeClassifier()
scores_2 = cross_val_score(model_1, X_train, y_train, cv = 5)
scores_2.mean()

0.7886363636363637

In [64]:
model_2 = BaggingClassifier(
    estimator = DecisionTreeClassifier(),
    n_estimators = 100,
    max_samples = 0.8,
    oob_score = True,
    random_state = 5)

scores_3 = cross_val_score(model, X_train, y_train, cv = 5) 
scores_3.mean()

0.8623251748251748

In [65]:
from sklearn.ensemble import RandomForestClassifier

scores_4 = cross_val_score(RandomForestClassifier(), X_train, y_train, cv = 5)
scores_4.mean()

0.8706487956487956